In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp parameter

# Parameter

Define parameter for meta-data, so algorithm can do cross section and mutation accordingly.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
import numpy as np
%pylab inline

In [ ]:
#export
import abc
from random import random, randint, normalvariate as randn
from math import log10

class Parameter:
    "Base class for parameter."
    def __init__(self, values, name=None, default_value=None):
        if name is None:
            self.name = str(self.__class__.__name__)
        else:
            self.name = str(name)
        self.set_values(values)
        if default_value is None:
            self.default_value = self.get_rand_value()
        else:
            self.default_value = default_value

    def __repr__(self):
        return f'{self.__class__.__name__}(values={self.values}, name="{self.name}", default_value={self.default_value})'

    @abc.abstractmethod
    def set_values(self, values):
        raise NotImplementedError('Not implemented')

    @abc.abstractmethod
    def get_rand_value(self, *args, **kwargs):
        raise NotImplementedError('Not implemented')

    def __next__(self):
        return self.get_rand_value()
    
    @abc.abstractmethod
    def is_valid(self, value, fix=False):
        raise NotImplementedError('Not implemented')

In [ ]:
#export
class OrderedParameter(Parameter):
    "Base class for parameter that is ordered, such as numerical parameters."
    @abc.abstractmethod
    def get_value(self, ratio=0):
        raise NotImplementedError('Not implemented')

In [ ]:
#export
class IntegerParameter(OrderedParameter):
    "Integer pramameter with a range."
    def set_values(self, values):
        if len(values) != 2:
            raise ValueError(
                f"values should have and only have two elements as lower and upper bound of the value"
            )
        self.values = sorted(values)

    def get_rand_value(self):
        return randint(*self.values)

    def get_value(self, ratio=0):
        if ratio > 1: ratio = 1
        if ratio < 0: ratio = 0
        return self.values[0] + round(
            (self.values[1] - self.values[0]) * ratio)
    
    def is_valid(self, value, fix=False):
        if value>=self.values[0]:
            if value <= self.values[1]:
                if fix:
                    return value
                else:
                    return True
            else:
                if fix:
                    return self.values[1]
                else:
                    return False
        else:
            if fix:
                return self.values[0]
            else:
                return False

In [ ]:
intp = IntegerParameter([0, 10])
assert intp.get_value(0.53) == 5
assert intp.is_valid(-2.3) is False
assert intp.is_valid(10.3, fix=True)==10

In [ ]:
#export
import bisect
class InListNumericParameter(OrderedParameter):
    """Numerical pramameter with a known set of values, and the values are ordered.
    Otherwise, it becomes a categorical parameter."""
    def set_values(self, values):
        self.values = sorted(values)
        self._len = len(self.values)

    def get_rand_value(self):
        return self.values[randint(1, self._len) - 1]

    def get_value(self, ratio=0):
        if ratio > 1: ratio = 1
        if ratio < 0: ratio = 0
        return self.values[round(self._len * ratio)]
    
    def is_valid(self, value, fix=False):
        if value in self.values:
            if fix:
                return value
            else:
                return True
        else:
            if fix:
                idx = bisect.bisect(self.values, value)
                idx = idx-1 if idx==self._len else idx
                return self.values[idx]
            else:
                return False

In [ ]:
inlistp = InListNumericParameter(range(30))
assert inlistp.get_value(0.5) == 15
inlistp.get_rand_value()
assert inlistp.is_valid(-2.3) is False
assert inlistp.is_valid(10.3, fix=True)==11

In [ ]:
#export
class FloatParameter(OrderedParameter):
    "Floating number parameter with a range."

    def set_values(self, values):
        if len(values) != 2:
            raise ValueError(
                f"values should have and only have two elements as lower and upper bound of the value"
            )
        self.values = sorted(values)
        self._range = self.values[1] - self.values[0]
        self._left = self.values[0]

    def get_rand_value(self, a=None, b=None):
        if a is None or b is None:
            return random() * self._range + self._left
        else:
            return random() * abs(a - b) + a if a < b else b

    def get_value(self, ratio=0):
        if ratio > 1: ratio = 1
        if ratio < 0: ratio = 0
        return self._range * ratio + self._left

    def is_valid(self, value, fix=False):
        if value >= self.values[0]:
            if value <= self.values[1]:
                if fix:
                    return value
                else:
                    return True
            else:
                if fix:
                    return self.values[1]
                else:
                    return False
        else:
            if fix:
                return self.values[0]
            else:
                return False

In [ ]:
floatp = FloatParameter([0, 10])
assert np.isclose(floatp.get_value(0.53), 5.3)
assert floatp.is_valid(-2.3) is False
assert floatp.is_valid(10.3, fix=True)==10

In [ ]:
#export
class LogFloatParameter(OrderedParameter):
    """Floating number parameter with a range, but the sampling is in a logrithmic scale.
    So lower paramter range is sampled more frequentyly than higher range.
    
    - Note: the parameter range must be positive, as `log` of negative number is not a real number.
    """
    def __init__(self, values, name=None, default_value=None):
        super().__init__(values, name, default_value=default_value)

    def set_values(self, values):
        if len(values) != 2:
            raise ValueError(
                f"values should have and only have two elements as lower and upper bound of the value"
            )
        self.values = sorted(values)
        self._left = log10(self.values[0])
        self._right = log10(self.values[1])
        self._range = self._right - self._left

    def get_rand_value(self, a=None, b=None):
        if a is None or b is None:
            return 10**(random() * self._range + self._left)
        else:
            a = log10(a)
            b = log10(b)
            return 10**(random() * abs(a - b) + a if a < b else b)

    def get_value(self, ratio=0):
        if ratio > 1: ratio = 1
        if ratio < 0: ratio = 0
        a = self._left
        b = self._right
        return 10**(ratio * abs(a - b) + a if a < b else b)

    def is_valid(self, value, fix=False):
        if value >= self.values[0]:
            if value <= self.values[1]:
                if fix:
                    return value
                else:
                    return True
            else:
                if fix:
                    return self.values[1]
                else:
                    return False
        else:
            if fix:
                return self.values[0]
            else:
                return False

In [ ]:
logp = LogFloatParameter([1, 100])
assert logp.get_value(0.5) == 10
assert logp.is_valid(-2.3) is False
assert logp.is_valid(103, fix=True)==100
logp.get_rand_value(1, 10)

`LogFloatParameter` samples values uniformly in a log scale.
The upper and lower bounds should both be positive.
The lower end will be sampled more than the higher end, as shown below:

In [ ]:
plot(sorted([logp.get_rand_value() for i in range(1000)]))

In [ ]:
#export
class CategoricalParameter(Parameter):
    "Categorical parameter"
    def set_values(self, values):
        self.values = list(set(values))
        self._len = len(self.values)

    def get_rand_value(self):
        return self.values[randint(1, self._len) - 1]
    
    def is_valid(self, value, fix=False):
        if value in self.values:
            if fix:
                return value
            else:
                return True
        else:
            if fix:
                return self.values[randint(1, self._len) - 1]
            else:
                return False

In [ ]:
catp = CategoricalParameter(['a', 'b', 'c'])
assert catp.get_rand_value() in ['a', 'b', 'c']
assert catp.is_valid('dd') is False
assert catp.is_valid('a')
assert catp.is_valid(103, fix=True) in 'abc'

In [ ]:
#export
class BooleanParameter(Parameter):
    "Boolean parameter"
    def __init__(self, values=(False, True), name=None, default_value=None):
        super().__init__(values, name=name, default_value=default_value)
        
    def set_values(self, values):
        if len(values)!=2:
            raise ValueError('Boolean parameter should have and only have two values.')
        self.values = list(values)

    def get_rand_value(self):
        return self.values[randint(0, 1)]
    
    def is_valid(self, value, fix=False):
        if value in self.values:
            if fix:
                return value
            else:
                return True
        else:
            if fix:
                return self.values[randint(0, 1)]
            else:
                return False

In [ ]:
boolp = BooleanParameter(name='bool')
assert boolp.get_rand_value() in [True, False]
assert boolp.is_valid('1') is False
assert boolp.is_valid(False)
assert boolp.is_valid(103, fix=True) in [True, False]

In [ ]:
#export
class CallableParameter(Parameter):
    """The values of the parameter is a callable. When execute the values attribute,
    the callable will return the possible value of the parameter."""
    def set_values(self, values):
        if callable(values):
            self.values = values
        else:
            raise ValueError('values need to be a callable object.')

    def get_rand_value(self, *args, **kwargs):
        return self.values(*args, **kwargs)

In [ ]:
from functools import partial
callp = CallableParameter(partial(randn, 100, 1))
callp.get_rand_value()

## How to use these parameter classes
In short, because `__next__` method is defined for the parameter classes, it can be treat as a generator. A typical example of these classes can be as follows:

In [ ]:
params = [LogFloatParameter([0.1,1000],'C'),
          CategoricalParameter(['poly', 'rbf', 'sigmoid'],"kernel"),
          LogFloatParameter([1e-6,1e6],'gamma')
         ]

In [ ]:
{p.name:next(p) for p in params}

In [ ]:
#hide
notebook2script()